In [1]:
import pandas as pd
import numpy as np
import geopandas as gpd
from functools import reduce
from geopandas import GeoDataFrame

#### We only need 2019 ipums data

In [2]:
hbd = pd.read_csv('./original_data/est_commuters_HBD.csv', index_col=0)
ipums = pd.read_csv('./original_data/commuter_origin_counts.csv',index_col=0)
ipums = ipums[ipums['YEAR']==2019].reset_index(drop=True).drop(['YEAR'],axis=1)

#### We will only visualize common transmodes of hbd & ipums

In [3]:
transmode = list(set(hbd.TransMode.unique()) & set(ipums.TransMode.unique()))
print('In Both:', transmode)
print('Left in IPUMS:', set(ipums.TransMode.unique())-set(transmode))
ipums = ipums[ipums['TransMode'].isin(transmode)].reset_index(drop=True)
ipums.shape

In Both: ['CommuterRail', 'Subway', 'AutoOccupants', 'Bicycle', 'Bus', 'Ferry']
Left in IPUMS: {'Walk', 'Other', 'WFH'}


(4185, 6)

#### Now the df.shape indicates there is some missing hour. For better visualization, we want to make sure the dataset is complete in time.

In [4]:
puma_mode = ipums[["STATEFIP","PUMA","PUMA_NAME","TransMode"]].drop_duplicates().reset_index(drop=True)
puma_mode['tmp'] = 1
hour = pd.DataFrame(range(24),columns=['Hour'])
hour['tmp'] = 1
tmp = reduce(lambda left,right:pd.merge(left,right,on='tmp',how='inner'),[puma_mode,hour]).drop('tmp',axis=1)
print("The complete dataset should have rows:", len(puma_mode)*24)
ipums_ = tmp.merge(right=ipums,on=['STATEFIP','PUMA',"PUMA_NAME",'TransMode','Hour'],how='outer')
ipums_['Estimated_Commuters'] = ipums_['Estimated_Commuters'].fillna(0).astype(int)
ipums_.shape

The complete dataset should have rows: 19608


(19608, 6)

In [5]:
ipums_reshape = ipums_.pivot_table(values='Estimated_Commuters', index=['STATEFIP','PUMA','PUMA_NAME','Hour'], columns='TransMode').fillna(0).astype(int).reset_index()
ipums_reshape['ALL'] = ipums_reshape['AutoOccupants'] + ipums_reshape['Bicycle'] + ipums_reshape['Bus'] + \
                       ipums_reshape['CommuterRail'] + ipums_reshape['Ferry'] + ipums_reshape['Subway']
ipums_reshape

TransMode,STATEFIP,PUMA,PUMA_NAME,Hour,AutoOccupants,Bicycle,Bus,CommuterRail,Ferry,Subway,ALL
0,9,100,"Danbury, Ridgefield, Bethel, Brookfield, New F...",0,69,0,0,0,0,0,69
1,9,100,"Danbury, Ridgefield, Bethel, Brookfield, New F...",1,0,0,0,0,0,0,0
2,9,100,"Danbury, Ridgefield, Bethel, Brookfield, New F...",2,0,0,0,0,0,0,0
3,9,100,"Danbury, Ridgefield, Bethel, Brookfield, New F...",3,0,0,0,0,0,0,0
4,9,100,"Danbury, Ridgefield, Bethel, Brookfield, New F...",4,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
6187,50,400,"Southwest Vermont--Rutland, Bennington & Addis...",19,0,0,0,0,0,0,0
6188,50,400,"Southwest Vermont--Rutland, Bennington & Addis...",20,0,0,0,0,0,0,0
6189,50,400,"Southwest Vermont--Rutland, Bennington & Addis...",21,0,0,0,0,0,0,0
6190,50,400,"Southwest Vermont--Rutland, Bennington & Addis...",22,0,0,0,0,0,0,0


In [6]:
### agg by hour
ipums_reshape_alltime = ipums_reshape.groupby(by=['STATEFIP','PUMA','PUMA_NAME']).\
                                      agg({"AutoOccupants":"sum",
                                           "Bicycle":"sum",
                                           "Bus":"sum",
                                           "CommuterRail":"sum",
                                           "Ferry":"sum",
                                           "Subway":"sum",
                                           "ALL":"sum"}).reset_index()
### this step is only for the convenience of spatial visualzation, replace zero with NaN
ipums_reshape_alltime[['AutoOccupants','Bicycle','Bus','CommuterRail','Ferry','Subway','ALL']] = \
ipums_reshape_alltime[['AutoOccupants','Bicycle','Bus','CommuterRail','Ferry','Subway','ALL']].replace(0, np.nan)
ipums_reshape_alltime

TransMode,STATEFIP,PUMA,PUMA_NAME,AutoOccupants,Bicycle,Bus,CommuterRail,Ferry,Subway,ALL
0,9,100,"Danbury, Ridgefield, Bethel, Brookfield, New F...",353.0,NaN,127.0,3060.0,NaN,395.0,3935
1,9,101,"Fairfield, New Canaan, Wilton, Weston & Easton...",701.0,NaN,NaN,6342.0,NaN,456.0,7499
2,9,102,Stamford & Greenwich Towns,1337.0,NaN,188.0,10157.0,NaN,379.0,12061
3,9,103,"Norwalk, Westport & Darien Towns",772.0,NaN,89.0,7849.0,NaN,781.0,9491
4,9,104,Bridgeport Town,262.0,NaN,84.0,1014.0,NaN,28.0,1388
...,...,...,...,...,...,...,...,...,...,...
253,44,103,Providence City,NaN,NaN,297.0,38.0,NaN,NaN,335
254,44,201,Central Rhode Island--Kent County--Warwick City,90.0,NaN,NaN,NaN,NaN,NaN,90
255,44,300,Southeast Rhode Island--Newport & Bristol Coun...,NaN,NaN,NaN,74.0,NaN,66.0,140
256,44,400,South Rhode Island--Washington County,157.0,NaN,NaN,121.0,NaN,30.0,308


#### Merge with puma spatial file

In [7]:
puma_boundary = gpd.read_file('./spatial_data/puma_boundary.geojson')
puma_boundary.rename({"statefip":"STATEFIP","state":"STATE","puma":"PUMA","name":"PUMA_NAME"},axis=1,inplace=True)
puma_boundary = puma_boundary[["STATEFIP","PUMA","PUMA_NAME","geometry"]]
puma_boundary[["STATEFIP","PUMA"]] = puma_boundary[["STATEFIP","PUMA"]].astype(int)
ipums_viz_alltime = puma_boundary.merge(ipums_reshape_alltime,on=["STATEFIP","PUMA"],how='outer')\
                                 .drop(['PUMA_NAME_y'],axis=1)\
                                 .rename({"PUMA_NAME_x":"PUMA_NAME"},axis=1)
ipums_viz_alltime

,STATEFIP,PUMA,PUMA_NAME,geometry,AutoOccupants,Bicycle,Bus,CommuterRail,Ferry,Subway,ALL
0,9,102,Stamford & Greenwich Towns PUMA,"MULTIPOLYGON (((-73.62324 40.98470, -73.62299 ...",1337.0,NaN,188.0,10157.0,NaN,379.0,12061.0
1,9,101,"Fairfield, New Canaan, Wilton, Weston & Easton...","MULTIPOLYGON (((-73.24417 41.22659, -73.24398 ...",701.0,NaN,NaN,6342.0,NaN,456.0,7499.0
2,9,100,"Danbury, Ridgefield, Bethel, Brookfield, New F...","MULTIPOLYGON (((-73.33157 41.47268, -73.33268 ...",353.0,NaN,127.0,3060.0,NaN,395.0,3935.0
3,9,103,"Norwalk, Westport & Darien Towns PUMA","MULTIPOLYGON (((-73.47606 41.04673, -73.47616 ...",772.0,NaN,89.0,7849.0,NaN,781.0,9491.0
4,9,105,"Stratford, Shelton, Trumbull, Newtown & Monroe...","MULTIPOLYGON (((-73.14335 41.15310, -73.14285 ...",322.0,NaN,NaN,1118.0,NaN,162.0,1602.0
...,...,...,...,...,...,...,...,...,...,...,...
414,42,701,"Lackawanna County--Scranton City, Dunmore, Old...","MULTIPOLYGON (((-75.68724 41.33922, -75.70139 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
415,42,3701,Adams & Franklin (Southeast) Counties PUMA,"MULTIPOLYGON (((-76.95505 39.85698, -76.95493 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
416,42,3702,Franklin County (Outside Washington Township &...,"MULTIPOLYGON (((-77.67176 40.28983, -77.66419 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN
417,42,1502,Beaver County (South) PUMA,"MULTIPOLYGON (((-80.15449 40.77267, -80.15450 ...",88.0,NaN,NaN,NaN,NaN,NaN,88.0


In [8]:
### Due to (multi)polygon geometry will make the size of file much larger 
### for the spatial visualization of 24-hour data, we will merge with the centroid of each puma to reduce size
puma_centroid = pd.read_csv('./spatial_data/puma_centroid.csv')
ipums_viz = ipums_reshape.merge(puma_centroid,on=["STATEFIP","PUMA"])
### this step is only for the convenience of spatial visualzation, replace zero with NaN
ipums_viz[['AutoOccupants','Bicycle','Bus','CommuterRail','Ferry','Subway','ALL']] = \
ipums_viz[['AutoOccupants','Bicycle','Bus','CommuterRail','Ferry','Subway','ALL']].replace(0, np.nan)
ipums_viz

,STATEFIP,PUMA,PUMA_NAME,Hour,AutoOccupants,Bicycle,Bus,CommuterRail,Ferry,Subway,ALL,lat,lon
0,9,100,"Danbury, Ridgefield, Bethel, Brookfield, New F...",0,69.0,NaN,NaN,NaN,NaN,NaN,69.0,41.405858,-73.454362
1,9,100,"Danbury, Ridgefield, Bethel, Brookfield, New F...",1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.405858,-73.454362
2,9,100,"Danbury, Ridgefield, Bethel, Brookfield, New F...",2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.405858,-73.454362
3,9,100,"Danbury, Ridgefield, Bethel, Brookfield, New F...",3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.405858,-73.454362
4,9,100,"Danbury, Ridgefield, Bethel, Brookfield, New F...",4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,41.405858,-73.454362
...,...,...,...,...,...,...,...,...,...,...,...,...,...
6187,50,400,"Southwest Vermont--Rutland, Bennington & Addis...",19,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.580522,-73.087118
6188,50,400,"Southwest Vermont--Rutland, Bennington & Addis...",20,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.580522,-73.087118
6189,50,400,"Southwest Vermont--Rutland, Bennington & Addis...",21,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.580522,-73.087118
6190,50,400,"Southwest Vermont--Rutland, Bennington & Addis...",22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,43.580522,-73.087118


#### Now, the dataframe is good for spatial visualiation and analysis.

In [9]:
### if we want to use the timebar function of kepler.gl, time column must be 'time' type
ipums_viz['Hour'] = pd.to_datetime(ipums_viz['Hour'], unit='h')
ipums_viz.to_csv('ipums_viz.csv',index=0)

In [10]:
ipums_viz_alltime = GeoDataFrame(ipums_viz_alltime, crs="EPSG:4326", geometry='geometry')
ipums_viz_alltime.to_file("ipums_viz_alltime.geojson", driver='GeoJSON')